# Retrieval
- 검색은 챗봇이 훈련 데이터 외부의 데이터로 응답을 보강하는 일반적인 기술입니다. 
- 이 섹션에서는 챗봇의 맥락에서 검색을 구현하는 방법을 다룰 것입니다. 
- 하지만 검색은 매우 미묘하고 깊은 주제임을 알아두는 것이 좋습니다 
- 문서의 다른 부분을 탐색하여 더 깊이 있는 내용을 살펴보도록 권장합니다!

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('../dot.env')

True

In [2]:
%pip install --upgrade --quiet langchain langchain-openai langchain-chroma beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

아래의 예제는 langsmith 공식문서의 내용을 바탕으로 답변하는 검색기 예제입니다.

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("Can LangSmith help test my LLM applications?")

docs

[Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip inst

In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know" and please answer in Korean:

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

아래의 prompt를 확인하면 document_chain에 들어가는 변수들 간의 관계를 알 수 있습니다.

특히 중요한 변수는 context와 messages입니다. 
context는 위에서 정의한 vectorstore에서 가져온 문서들을 말하며, 
messages는 사용자의 질문에 대한 답변을 만들어내는 변수입니다. 
즉, 사용자가 질문한 내용을 바탕으로 문서를 검색하고, 그 결과를 바탕으로 답변을 만들어내는 과정을 거쳐야 합니다.

In [6]:
question_answering_prompt

ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the user\'s questions based on the below context. \nIf the context doesn\'t contain any relevant information to the question, don\'t make something up and just say "I don\'t know" and please answer in Korean:\n\n<context>\n{context}\n</context>\n')), MessagesPlaceholder(variable_name='messages')])

In [7]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?")
        ],
    }
)

'LangSmith는 LLM 애플리케이션을 테스트하는 데 도움을 줄 수 있습니다.'

아래는 langsmith docs를 context로 주지 않았을 때의 답변입니다.

In [8]:
document_chain.invoke(
    {
        "context": [],
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?")
        ],
    }
)

'제가 알기로는 LangSmith는 LLM 애플리케이션을 테스트하는 데 도움을 줄 수 있습니다. 그러나 구체적인 세부 정보는 알 수 없습니다. 도움이 필요하시면 LangSmith에게 직접 문의해보시는 것이 좋을 것 같습니다.'

이제 위에서 생성한 doc chain과 검색기를 연결해보겠습니다.

In [26]:
# from typing import Dict

# from langchain_core.runnables import RunnablePassthrough

# # 'params' 사전에서 마지막 메시지의 내용을 추출하는 함수
# def parse_retriever_input(params: Dict):
#     return params["messages"][-1].content

# # 검색 체인을 설정합니다. 이 체인은 입력된 메시지를 처리하여 적절한 문서를 검색하고,
# # 그 결과를 document_chain에 전달하여 최종 답변을 생성합니다.
# ''' RunnablePassthrough.assign은 체인의 다음 단계로 데이터를 전달하는 데 사용됩니다.
#  document_chain의 앞단에 parse_retriever_input과 retriever를 연결하여 
#  마지막 메시지를 검색기에 전달하고, 그 결과를 document_chain에 전달하여 답변을 생성합니다.'''

# retrieval_chain = RunnablePassthrough.assign(
#     context=parse_retriever_input | retriever,  # 마지막 메시지를 검색기에 전달
# ).assign(
#     answer=document_chain,  # 검색 결과를 document_chain에 전달하여 답변 생성
# )

from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [27]:
retrieval_chain.__dict__

{'name': None,
 'first': RunnableAssign(mapper={
   context: RunnableLambda(parse_retriever_input)
            | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x115dd7290>)
 }),
 'middle': [],
 'last': RunnableAssign(mapper={
   answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
             context: RunnableLambda(format_docs)
           }), config={'run_name': 'format_inputs'})
           | ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the user\'s questions ba

In [28]:
retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?")
        ],
    }
)

{'messages': [HumanMessage(content='Can LangSmith help test my LLM applications?')],
 'context': [Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
  Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangC

# 쿼리 변환
- 우리의 검색 체인은 LangSmith에 대한 질문에 답할 수 있지만, 문제는 챗봇이 사용자와 대화식으로 상호작용하기 때문에 후속 질문을 처리해야 한다는 점입니다. 
- 현재 형태의 체인은 이 문제를 해결하기 어렵습니다. 
- 예를 들어, 원래 질문에 대한 후속 질문으로 "더 알려주세요!"와 같은 질문을 고려해보세요. 검색기에 해당 쿼리를 직접 전달하면 LLM 애플리케이션 테스트와 관련 없는 문서가 반환됩니다:

In [29]:
retriever.invoke("Tell me more!")

[Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
 Document(page_content='result.choices[0].message.contentpipeline("Hello, world!")# Out:  Hello there! How can I assist you today?import { OpenAI } from "openai";import { traceable } from "langsmith/traceable";import { wrapOpenAI } from "langsmith/wrappers";// Auto-trace LLM calls in-contextconst client = wrapOpenAI(new OpenAI());// Auto-trace this functionconst pipeline = traceable(async (user_input) => {    const result = await client.chat.completions.create({        messages: [{

In [30]:
from langchain_core.messages import AIMessage, HumanMessage

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)
#위 query_transform_prompt는 message 내의 내서를 바탕으로 쿼리를 만들어내는 체인입니다. 

query_transformation_chain = query_transform_prompt | chat

query_transformation_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
            AIMessage(
                content="Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)
#query_transformation_chain은 주어진 messages를 바탕으로 "LangSmith LLM application testing and evaluation"라는 검색기를 위한 쿼리를 만들었습니다.

AIMessage(content='"LangSmith LLM application testing and evaluation"', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 145, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_482d920018', 'finish_reason': 'stop', 'logprobs': None}, id='run-d6b39a1c-0e9e-404c-98c5-67472b06f864-0', usage_metadata={'input_tokens': 145, 'output_tokens': 10, 'total_tokens': 155})

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

#Runnablebranch -> if문 형식의 분기처리를 지원 if와 else는 따로 표기하지 않는듯?'
query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever, 대화가 하나이면 llm을 사용하지 않고 그냥 검색
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [41]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

'''
query_transforming_retriever_chain:
query_transform_prompt | chat | StrOutputParser() | retriever
그간의 메시지를 이용, 검색의 위한 쿼리를 생성하고 검색을 실행,
이후 document chain에 해당 내용을 전달, 답변을 생성합니다.
'''
conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

In [51]:
response = conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
        ]
    }
)
response['answer']


'Yes, LangSmith is a platform designed specifically for building production-grade LLM (Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Therefore, LangSmith can indeed help test your LLM applications.'

In [52]:
conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
            AIMessage(
                content=response['answer']
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)

{'messages': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes, LangSmith is a platform designed specifically for building production-grade LLM (Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Therefore, LangSmith can indeed help test your LLM applications.'),
  HumanMessage(content='Tell me more!')],
 'context': [Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
  Document(page_content='Skip to m